In [ ]:
import pandas as pd
import sklearn

In [28]:
train_idx = pd.read_json('Downloads/module-8-assignment-1-train-idx.json')
val_idx = pd.read_json('Downloads/module-8-assignment-1-validation-idx.json')

In [49]:
train_idx.columns

RangeIndex(start=0, stop=1, step=1)

In [6]:
club = pd.read_csv('Downloads/lending-club-data.csv')

/Users/sofia/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
club['bad_loans'].value_counts()

0    99457
1    23150
Name: bad_loans, dtype: int64

In [17]:
club['safe_loans'] = club['bad_loans'].apply(lambda loans: +1 if loans == 0 else -1)

In [18]:
club['safe_loans'].value_counts()

 1    99457
-1    23150
Name: safe_loans, dtype: int64

In [21]:
club = club.drop(['bad_loans'], axis=1)

In [22]:
target = ['safe_loans']
features = ['grade',                     # grade of the loan (categorical)
            'sub_grade_num',             # sub-grade of the loan as a number from 0 to 1
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'payment_inc_ratio',         # ratio of the monthly payment to income
            'delinq_2yrs',               # number of delinquincies
             'delinq_2yrs_zero',          # no delinquincies in last 2 years
            'inq_last_6mths',            # number of creditor inquiries in last 6 months
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'open_acc',                  # number of open credit accounts
            'pub_rec',                   # number of derogatory public records
            'pub_rec_zero',              # no derogatory public records
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
            'int_rate',                  # interest rate of the loan
            'total_rec_int',             # interest received to date
            'annual_inc',                # annual income of borrower
            'funded_amnt',               # amount committed to the loan
            'funded_amnt_inv',           # amount committed by investors for the loan
            'installment',               # monthly payment owed by the borrower
           ]


In [26]:
club_a = club[features + target].dropna()

In [27]:
club_a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122578 entries, 0 to 122606
Data columns (total 25 columns):
grade                    122578 non-null object
sub_grade_num            122578 non-null float64
short_emp                122578 non-null int64
emp_length_num           122578 non-null int64
home_ownership           122578 non-null object
dti                      122578 non-null float64
purpose                  122578 non-null object
payment_inc_ratio        122578 non-null float64
delinq_2yrs              122578 non-null float64
delinq_2yrs_zero         122578 non-null float64
inq_last_6mths           122578 non-null float64
last_delinq_none         122578 non-null int64
last_major_derog_none    122578 non-null int64
open_acc                 122578 non-null float64
pub_rec                  122578 non-null float64
pub_rec_zero             122578 non-null float64
revol_util               122578 non-null float64
total_rec_late_fee       122578 non-null float64
int_rate          

In [51]:
train = club_a.iloc[train_idx[0]]
val = club_a.iloc[val_idx[0]]

In [ ]:
#train&test data - balanced
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)


#one hot encoder
loans_data = risky_loans.append(safe_loans)

categorical_variables = []
for feat_name, feat_type in zip(loans_data.column_names(), loans_data.column_types()):
    if feat_type == str:
        categorical_variables.append(feat_name)

for feature in categorical_variables:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)

    # Change None's to 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data.remove_column(feature)
    loans_data.add_columns(loans_data_unpacked)

loans_data.column_names()

In [55]:
import sklearn.ensemble
import numpy as np

In [56]:
from sklearn.ensemble import GradientBoostingClassifier

In [57]:
GBC = GradientBoostingClassifier(max_depth=6, n_estimators=5)

In [58]:
target = club_a['safe_loans']
features = club_a.drop(['safe_loans'], axis=1)

In [60]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122578 entries, 0 to 122606
Data columns (total 24 columns):
grade                    122578 non-null object
sub_grade_num            122578 non-null float64
short_emp                122578 non-null int64
emp_length_num           122578 non-null int64
home_ownership           122578 non-null object
dti                      122578 non-null float64
purpose                  122578 non-null object
payment_inc_ratio        122578 non-null float64
delinq_2yrs              122578 non-null float64
delinq_2yrs_zero         122578 non-null float64
inq_last_6mths           122578 non-null float64
last_delinq_none         122578 non-null int64
last_major_derog_none    122578 non-null int64
open_acc                 122578 non-null float64
pub_rec                  122578 non-null float64
pub_rec_zero             122578 non-null float64
revol_util               122578 non-null float64
total_rec_late_fee       122578 non-null float64
int_rate          

In [59]:
GBC.fit(features, target)

ValueError: could not convert string to float: 'B'

In [ ]:
def find_word(a, target):
    index=0
    for word in a:
        if word == target:
            return index
        index += 1
    return -1

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

# these two lines are what allow your code to show up in a notebook!
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

In [1]:
import os
directory = os.getcwd()
print(directory)


/Users/sofia
